In [60]:
import requests
import pandas as pd

# API URL 설정
api_url = "http://openapi.airgwangsan.kr:8080/Gwangsan/getDustDataAPI?apiId=01"

# API 호출
response = requests.get(api_url)
# JSON 데이터를 가져오기
data = response.json()

In [61]:
data

{'response': {'resultCode': '01',
  'resultMsg': '당일 기준 데이터 입니다.',
  'totalCount': 16391,
  'dataType': 'JSON',
  'items': [{'place': '하남동 홈플러스 하남점',
    'PM1': '10.0',
    'PM10': '18.0',
    'PM2_5': '11.2',
    'CO2': '498.0',
    'TEMPERATURE': '32.8',
    'HUMIDITY': '34.6',
    'LATITUDE': '35.17861940',
    'LONGITUDE': '126.8066483',
    'COLLECTION_DATE': '2024-06-18 21:35:59.0'},
   {'place': '하남동 홈플러스 하남점',
    'PM1': '9.4',
    'PM10': '18.0',
    'PM2_5': '10.8',
    'CO2': '500.0',
    'TEMPERATURE': '32.8',
    'HUMIDITY': '35.0',
    'LATITUDE': '35.17861940',
    'LONGITUDE': '126.8066483',
    'COLLECTION_DATE': '2024-06-18 21:30:59.0'},
   {'place': '하남동 홈플러스 하남점',
    'PM1': '10.7',
    'PM10': '18.0',
    'PM2_5': '12.2',
    'CO2': '512.0',
    'TEMPERATURE': '33.4',
    'HUMIDITY': '34.6',
    'LATITUDE': '35.17861940',
    'LONGITUDE': '126.8066483',
    'COLLECTION_DATE': '2024-06-18 21:05:59.0'},
   {'place': '하남동 홈플러스 하남점',
    'PM1': '8.9',
    'PM10': '15.0

In [62]:
df = pd.DataFrame(data['response']['items'])
df_unique = df.drop_duplicates(subset=['place'])
df_unique

,place,PM1,PM10,PM2_5,CO2,TEMPERATURE,HUMIDITY,LATITUDE,LONGITUDE,COLLECTION_DATE
0,하남동 홈플러스 하남점,10.0,18.0,11.2,498.0,32.8,34.6,35.17861940,126.8066483,2024-06-18 21:35:59.0
133,소촌동 현성 오토텍,14.3,18.0,14.8,242.0,41.1,21.0,35.15502930,126.7882538,2024-06-18 21:30:59.0
263,신창동 해피키즈 어린이집,9.7,33.8,11.8,2134.0,34.8,40.8,35.19879315,126.8379488,2024-06-18 21:30:59.0
388,운남동 한사랑병원,7.4,18.0,8.6,411.0,40.3,22.5,35.17269000,126.8176300,2024-06-18 21:35:59.0
516,수완동 하바드 어린이집,8.0,21.6,9.6,419.0,38.5,24.9,35.20105000,126.8273400,2024-06-18 21:25:59.0
...,...,...,...,...,...,...,...,...,...,...
15862,월곡동 광산구 노인 복지관,6.9,22.0,8.0,370.0,40.1,22.9,35.17160030,126.8146973,2024-06-18 21:35:59.0
15979,복룡동 관두 경로당,14.1,22.0,16.1,413.0,37.5,26.6,35.10205000,126.7821600,2024-06-18 21:25:59.0
16108,신촌동 공항역,8.0,16.1,9.5,420.0,25.5,51.7,35.11037400,126.8819910,2024-06-18 21:30:59.0
16334,월계동 KB 국민은행,5.8,20.4,7.4,996.0,33.6,47.1,35.21361160,126.8458252,2024-06-18 21:35:59.0


In [63]:
df_unique.rename(columns={'LATITUDE':'lat', 'LONGITUDE':'lot'})

,place,PM1,PM10,PM2_5,CO2,TEMPERATURE,HUMIDITY,lat,lot,COLLECTION_DATE
0,하남동 홈플러스 하남점,10.0,18.0,11.2,498.0,32.8,34.6,35.17861940,126.8066483,2024-06-18 21:35:59.0
133,소촌동 현성 오토텍,14.3,18.0,14.8,242.0,41.1,21.0,35.15502930,126.7882538,2024-06-18 21:30:59.0
263,신창동 해피키즈 어린이집,9.7,33.8,11.8,2134.0,34.8,40.8,35.19879315,126.8379488,2024-06-18 21:30:59.0
388,운남동 한사랑병원,7.4,18.0,8.6,411.0,40.3,22.5,35.17269000,126.8176300,2024-06-18 21:35:59.0
516,수완동 하바드 어린이집,8.0,21.6,9.6,419.0,38.5,24.9,35.20105000,126.8273400,2024-06-18 21:25:59.0
...,...,...,...,...,...,...,...,...,...,...
15862,월곡동 광산구 노인 복지관,6.9,22.0,8.0,370.0,40.1,22.9,35.17160030,126.8146973,2024-06-18 21:35:59.0
15979,복룡동 관두 경로당,14.1,22.0,16.1,413.0,37.5,26.6,35.10205000,126.7821600,2024-06-18 21:25:59.0
16108,신촌동 공항역,8.0,16.1,9.5,420.0,25.5,51.7,35.11037400,126.8819910,2024-06-18 21:30:59.0
16334,월계동 KB 국민은행,5.8,20.4,7.4,996.0,33.6,47.1,35.21361160,126.8458252,2024-06-18 21:35:59.0


In [64]:
def pm_to_color(pm10, pm2_5):
    if pm10 >= 151 and pm2_5 >= 76:
        return 'red'
    elif pm10 >= 81 and pm2_5 >= 36:
        return 'yellow'
    elif pm10 >= 31 and pm2_5 >= 16:
        return 'green'
    else:
        return 'blue'

In [65]:
import folium
import pandas as pd

# 지도의 초기 위치 설정
mymap = folium.Map(location=[35.17861940, 126.8066483], zoom_start=12)

# 각 좌표에 대해 마커를 지도에 추가합니다.
for index, row in df_unique.iterrows():
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']
    place = row['place']
    pm10 = float(row['PM10'])
    pm2_5 = float(row['PM2_5'])

        
    if pm10 >= 151 and pm2_5 >= 76:
        folium.Marker(
            location=[latitude, longitude],
            popup=place,
            icon=folium.Icon(color='red', icon='cloud')
        ).add_to(mymap)
    elif pm10 >= 81 and pm2_5 >= 36:
            folium.Marker(
            location=[latitude, longitude],
            popup=place,
            icon=folium.Icon(color='orange', icon='cloud')
        ).add_to(mymap)
    elif pm10 >= 31 and pm2_5 >= 16:
            folium.Marker(
            location=[latitude, longitude],
            popup=place,
            icon=folium.Icon(color='green', icon='cloud')
        ).add_to(mymap)
    else:
            folium.Marker(
            location=[latitude, longitude],
            popup=place,
            icon=folium.Icon(color='blue', icon='cloud')
        ).add_to(mymap)
      


# 지도 출력
mymap

